## csv 파일 excel로 변환

In [53]:
import os
import pandas as pd

path = "/Users/jmyeong/Desktop/spam"
kisa_path = "/Users/jmyeong/Desktop/kisa-server"
# file_lst = os.listdir(path)

file_lst = os.listdir(path)

for file in file_lst:
    if file[4:6] != "11":
        continue
    filepath = path + '/' + file
    file_sub_lst = os.listdir(filepath)
    for sub in file_sub_lst:
        if sub == ".DS_Store":
            continue

        try : 
            f = filepath + '/' + sub
            a = pd.read_csv(f, delimiter='|', header=None)
            a.rename(columns = {0: "구분", 1 : "수신시간", 2: "신고시간", 3:"스팸유형", 4: "광고유형", 5: "전문내용"}, inplace=True)
            a["원발신번호"] = ""
            a["회신번호"] = ""
            a["신고인번호"] = ""
            a = a[["구분","수신시간","신고시간","스팸유형","광고유형","원발신번호","회신번호","신고인번호","전문내용"]]
            a = a[a["구분"] == "민원"]
            directory = kisa_path+'/'+file[:6]+'/'+file[6:]

            if not os.path.exists(directory):
                os.makedirs(directory)
            a.to_excel(directory+'/'+sub[:2]+".xls", index=False)
        except Exception as e:
            continue

/var/folders/jp/nrj6zk491wv9npgslcl58jf00000gn/T/ipykernel_17435/1379117821.py:32: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  a.to_excel(directory+'/'+sub[:2]+".xls", index=False)
/var/folders/jp/nrj6zk491wv9npgslcl58jf00000gn/T/ipykernel_17435/1379117821.py:32: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While th

/var/folders/jp/nrj6zk491wv9npgslcl58jf00000gn/T/ipykernel_17435/1379117821.py:32: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  a.to_excel(directory+'/'+sub[:2]+".xls", index=False)
/var/folders/jp/nrj6zk491wv9npgslcl58jf00000gn/T/ipykernel_17435/1379117821.py:32: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While th

In [54]:
import os
import pandas as pd

path = "/Users/jmyeong/Desktop/kisa-server/"
file_lst = os.listdir(path)

def receiveTime_exception_find(text):
    try :
        if  text.startswith("010",0,4):
            text = ''
            return text
        else:
            if len(text) == 16 :
                text = text + ":00"
            text = text.replace(".","-")
            return text
    except:
        text = ''
        return text


def reportTime_exception_find(text):
    try :
        if text.startswith("2022",0,4) | text.startswith("2021",0,4) | text.startswith("2020",0,4):
            text = text.replace(".","-")
            return text
        else:
            text = ''
            return text
    except:
        text = ''
        return text

def content_exception_find(text):
    try :
        if text.startswith("2022",0,4) | text.startswith("2021",0,4) | text.startswith("2020",0,4) | text.startswith("010",0,4):
            text = ''
            return text
        return text
    except AttributeError as e:
        print(e)
        text = ''
        return text
    
for file in file_lst:
    if file != "202211":
        continue
        
    df = pd.DataFrame()
    filepath = path + '/' + file
    file_sub_lst = os.listdir(filepath)
    for sub in file_sub_lst:
        if sub == ".DS_Store":
            continue
        f = filepath + '/' + sub
        real_file = os.listdir(f)
        for r in real_file:
            rf = f + '/' + r
            if r == ".DS_Store":
                continue
            print(rf)
            try:
                data = pd.read_excel(rf).drop(["구분","원발신번호","회신번호","신고인번호"], axis=1)
                data["수신시간"] = data["수신시간"].apply(lambda x : receiveTime_exception_find(x))
                data["신고시간"] = data["신고시간"].apply(lambda x : reportTime_exception_find(x))
                data["전문내용"] = data["전문내용"].apply(lambda x : content_exception_find(x))
                data = data[data["수신시간"] != '']
                data = data[data["신고시간"] != '']
                data = data[data["전문내용"] != '']
                df = pd.concat([df, data])
            except AssertionError as e:
                print(rf)
    
    df.rename(columns = {"수신시간":'receive_time', "신고시간" :'report_time', "스팸유형" : 'spam_type', "광고유형":'ad_type', "전문내용": 'content'},inplace=True)
#     df.to_parquet(f"./{file}.parquet")
    df.to_parquet(f"./parquet_{file}01")
    print(f"{file} 완료")
           

/Users/jmyeong/Desktop/kisa-server//202211/01/11.xls
/Users/jmyeong/Desktop/kisa-server//202211/01/05.xls
/Users/jmyeong/Desktop/kisa-server//202211/01/04.xls
/Users/jmyeong/Desktop/kisa-server//202211/01/10.xls
/Users/jmyeong/Desktop/kisa-server//202211/01/06.xls
/Users/jmyeong/Desktop/kisa-server//202211/01/12.xls
/Users/jmyeong/Desktop/kisa-server//202211/01/13.xls
/Users/jmyeong/Desktop/kisa-server//202211/01/07.xls
/Users/jmyeong/Desktop/kisa-server//202211/01/03.xls
/Users/jmyeong/Desktop/kisa-server//202211/01/17.xls
/Users/jmyeong/Desktop/kisa-server//202211/01/16.xls
/Users/jmyeong/Desktop/kisa-server//202211/01/02.xls
/Users/jmyeong/Desktop/kisa-server//202211/01/14.xls
/Users/jmyeong/Desktop/kisa-server//202211/01/00.xls
/Users/jmyeong/Desktop/kisa-server//202211/01/01.xls
/Users/jmyeong/Desktop/kisa-server//202211/01/15.xls
/Users/jmyeong/Desktop/kisa-server//202211/01/18.xls
/Users/jmyeong/Desktop/kisa-server//202211/01/19.xls
/Users/jmyeong/Desktop/kisa-server//202211/01/

In [43]:
!pwd

/Users/jmyeong/Desktop/jmyeong/spamAnalysis


In [146]:
df1 = pd.read_parquet("./spam/202204.parquet")

In [117]:
# df1[df1["report_time"] == '']
df1 = df1.loc[(df1["report_time"] != ''), :]

In [147]:
df1.loc[(df1["report_time"] != ''), :]

,receive_time,report_time,spam_type,ad_type,content
0,2022-04-01 12:18,2022-04-03 11:59:47,문자,기타,"[Web발신](광고)OW강남,송파 주부 점심시간30분 64,300원bit.ly/ne..."
1,2022-03-21 13:45,2022-04-03 11:58:21,문자,도박,[Web발신](광고)도신피[씨] 울산병원부근신규오픈했습니다!!^^놀러오세용~~무료거...
2,2022-04-02 14:04,2022-04-03 11:58:10,MMS,기타,[Web발신](광고)꿈은 이루어진다아무나 참여가능한손쉬운 방식 입니다▼▼▼▼하루 2...
3,2022-04-01 09:31,2022-04-03 11:57:39,문자,기타,[Web발신](광고)리드투자자산 open.kakao.com/o/gwH6ru7d 매수...
4,2022-04-03 11:45,2022-04-03 11:57:15,문자,기타,[Web발신][화이트]4월 첫 째주 일요일10 + 3 이벤트 진행 중입니다.쪽지 또...
...,...,...,...,...,...
518,2022-04-24 16:28,2022-04-25 08:06:35,MMS,도박,[Web발신](광고)계좌회복 프로젝트 1탄으로 공개한대한제당우 3연속 상한가 드셨죠...
519,2022-04-25 08:05,2022-04-25 08:06:27,MMS,불법대출,[Web발신](광고)안녕하세요.귀하께서는「소상공인 경영안정지원자금 및 카드수수료 지...
520,2022-04-24 17:39,2022-04-25 08:05:21,MMS,기타,"(광고)세력과 개미의 싸움?이제 그만 세력과 싸우시죠여러분들이 지금까지 접한 ""정보..."
521,2022-04-24 17:32,2022-04-25 08:03:56,MMS,기타,[Web발신](광고)세력과 개미의 싸움?이제 그만 세력과 싸우시죠여러분들이 지금까지...


In [29]:
for i in ['01','02','03','04','n']: 
    f = open(f"./spam_data_1y/kisa_req_result_{i}.csv",'r',encoding="CP949")
    fw = open(f"./spam_data_1y/kisa_result_{i}.csv", 'w',encoding="CP949")
    lines = f.readlines()
    for line in lines:
        info = line.split("|",5)
        if "." in info[2]:
            info[2] = info[2].replace(".","-") 
        if "|" in info[5]:
            info[5] = info[5].replace("|","")
        del info[0]
        fw.write("|".join(info))
    
    f.close()
    fw.close()


In [2]:
import pandas as pd

a = pd.read_csv("./spam_data_1y/kisa_result_01.csv", sep='|',encoding='CP949', header=None) # ,names=['a','b','c','d','f','g','h','i','j','k','l','m','n','o']
b = pd.read_csv("./spam_data_1y/kisa_result_02.csv", sep='|',encoding='CP949', header=None) # ,error_bad_lines=False
c = pd.read_csv("./spam_data_1y/kisa_result_03.csv", sep='|',encoding='CP949', header=None)
d = pd.read_csv("./spam_data_1y/kisa_result_04.csv", sep='|',encoding='CP949', header=None)
e = pd.read_csv("./spam_data_1y/kisa_result_n.csv", sep='|',encoding='CP949', header=None)

In [3]:
df = pd.concat([a,b,c,d,e])

In [4]:
df.rename(columns = {0:'receive_time', 1 :'report_time', 2 : 'spam_type', 3:'ad_type', 4: 'content'},inplace=True)


In [34]:
df.to_parquet("./year_spam.parquet")